In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
#------------------
import stochoptim.stochprob.network_design.network_design_problem as ndp
import stochoptim.stochprob.network_design.network_design_solution as nds
import stochoptim.stochprob.network_design.network_design_uncertainty as ndu

### Generate Problem Parameters

In [ ]:
n_origins = 2
n_destinations = 2
n_intermediates = 4
#------------
param = ndp.generate_random_parameters(n_origins, n_destinations, n_intermediates)
param

### Instantiate Problem

In [ ]:
network_problem = ndp.NetworkDesign(param)
network_problem

### Instantiate Problem Uncertainty

#### Lognormal Demand

In [ ]:
n_scenarios = 10
distribution = 'lognormal'
mean = 1
std = 1
#------------------
network_uncertainty = ndu.Demands(n_scenarios=n_scenarios, 
                                  n_commodities=n_destinations*n_origins, 
                                  distribution=distribution,
                                  mean=mean, 
                                  std=std)
network_uncertainty

#### Uniform Demand

In [ ]:
n_scenarios = 10
distribution = "uniform"
lb = 5
ub = 20
corr = 0.4
#-------------------
network_uncertainty = ndu.Demands(n_scenarios=n_scenarios,
                                  n_commodities=n_origins*n_destinations, 
                                  distribution=distribution,
                                  lb=lb, 
                                  ub=ub, 
                                  corr=corr)
network_uncertainty

### Build scenario tree

In [ ]:
# get only scenarios of index [1, 5]
scenario_tree = network_uncertainty.get_scenario_tree([1, 5])
scenario_tree

In [ ]:
# get all scenarios
scenario_tree = network_uncertainty.get_scenario_tree()
scenario_tree

### Solve the problem on the scenario tree

In [ ]:
network_solution = network_problem.solve(scenario_tree)
network_solution

### Plot optimal solution

In [ ]:
network_solution.plot_network()

In [ ]:
network_solution.plot_network(scen_index=0)